# Preprocess for training and test set

Sentinel-2 images need to be processed to correct invalid data, mask the clouds, derive NDVI index, superresolve, coregister to reduce small offsets between acquisitions, export the numpy arrays as compressed arrays. Geotiff images have also been produced just for a check.
Functions defined for the main tasks are applied both for the training and test set. The only differences between them are the number of images and the way to access them.

In [3]:
# Jupyter notebook related
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/opt/conda/envs/eurodatacube-gpu-0.24.5/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
# Built-in modules
import os
import sys
import json
import datetime as dt
from typing import Tuple, List

# Basics of Python data handling and visualization
import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm.auto import tqdm

# Module for GeoDB
from xcube_geodb.core.geodb import GeoDBClient

# Imports from eo-learn and sentinelhub-py
from sentinelhub import CRS, BBox, SHConfig, DataCollection
from eolearn.core import (FeatureType,
                          EOPatch, 
                          EOTask, 
                          LinearWorkflow, 
                          EOExecutor, 
                          LoadTask,
                          SaveTask)
from eolearn.io import GeoDBVectorImportTask, SentinelHubInputTask
from eolearn.geometry import VectorToRaster
from eolearn.coregistration import ThunderRegistration
from eolearn.io import ExportToTiff
from eolearn.core import AddFeature

# Other libraries for image processing
from skimage.registration import phase_cross_correlation
from skimage.transform import resize, warp
import scipy.ndimage
from registration import CrossCorr
register = CrossCorr()

/opt/conda/envs/eurodatacube-gpu-0.24.5/lib/python3.8/site-packages/bolt/utils.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  from collections import Iterable
/opt/conda/envs/eurodatacube-gpu-0.24.5/lib/python3.8/site-packages/bolt/factory.py:17: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args = inspect.getargspec(func)


### Connection parameters 

In [ ]:
sh_config = SHConfig()

sh_config.sh_client_id = '...'
sh_config.sh_client_secret = '...'

geodb_client_id = '...'
geodb_client_secret = '...'

client = GeoDBClient()

EOPATCHES_PATH = 'eopatches'
EOPATCHES_TRAIN_PATH = f'{EOPATCHES_PATH}/train/'
EOPATCHES_TEST_PATH = f'{EOPATCHES_PATH}/test/'

SUBMISSION_DIR = 'submission'
if not os.path.exists(SUBMISSION_DIR):
    os.makedirs(SUBMISSION_DIR)

# DO NOT CHANGE THIS
GEODB_DATABASE = 'geodb_0e5d743f-2134-4561-8946-a073b039176f'

### Area of Interest

In [ ]:
bboxes = client.get_collection('ai4eo_bboxes', database=GEODB_DATABASE)

### Sentinel-2 time-series

In [ ]:
# Download parameters - DO NOT CHANGE
S2_TIME_INTERVAL = ('2019-03-01','2019-09-01')

S2_RESOLUTION = 10  # metres
S2_MAXCC = 0.5
S2_TIME_DELTA = 120
THRES = 200

MAX_THREADS = 5

# process parameters
MIN_CLOUD_PROB = 19

### Definition of the main functions

##### Function for the interpolation of pixels along the time period in case of cloudy images

In [3]:
def interpolate(ndvi_in, valid_in, n_frames):
    x = np.linspace(0,n_frames-1,n_frames).astype(int)    
    ndvi_tmp = np.zeros((n_frames, 500, 500), dtype=np.int)

    #for every pixel along the time series
    for r in range(500):
        for c in range(500):
            
            # list of values for each pixel stored as ndvi2
            ndvi2 = ndvi_in[:,r,c].astype(int)   

            # if there is at least one invalid pixel then interpolate
            if sum(valid_in[:,r,c]) > 0:
                
                invalid = valid_in[:,r,c].nonzero()[0]  
                ndvi_ma = np.ma.masked_array(ndvi_in[:,r,c],valid_in[:,r,c]).compressed()
                x_ma = np.ma.masked_array(x, valid_in[:,r,c]).compressed()
                ndvi_new = np.interp(invalid, x_ma, ndvi_ma)     
                               
                for a in invalid:
                    pos = np.where(invalid == a)
                    ndvi2[a] = ndvi_new[pos].astype(int)

            ndvi_tmp[:,r,c] = ndvi2
     
    return ndvi_tmp

##### Function for the detection of the best reference cloud-free image

In [4]:
# choose of the best reference image as completely cloud-free starting from the last
def find_ref(clm_tmp, n_frames):
    for t in range(n_frames-1, -1, -1): 
        if np.sum(clm_tmp[t,...]) == 0:
            break
    return t

##### Function for the coregistration and the images superresolution

In [5]:
# coregistration task use the Cross-Correlation function of scikit-image
def coreg(ndvi_ds, tref, n_frames):
    #superres and coregister images
    reference = ndvi_ds[tref,...]
    
    ref = scipy.ndimage.zoom(reference, 4, order=3)
    imgs = [scipy.ndimage.zoom(ndvi_ds[s1,...], 4, order=3) for s1 in range(n_frames)]
    model = register.fit(imgs, reference=ref)

    shifted_tmp = model.transform(imgs)
    shifted_img = shifted_tmp.toarray()

    # consider the original image in case of wrong shifts
    for s2 in range(n_frames):
        dx = abs(int(str(model.transformations[(s2,)]).replace("Displacement(delta=[","").replace("])","").split(",")[0]))
        dy = abs(int(str(model.transformations[(s2,)]).replace("Displacement(delta=[","").replace("])","").split(",")[1]))
        if (dx > 10) | (dy > 10):
            #print(s2, dx, dy)
            shifted_img[s2,...] = scipy.ndimage.zoom(ndvi_ds[s2,...], 4, order=3)
    return shifted_img

### Preprocessing of the training set

In [ ]:
ndvi_hi_sd = np.zeros((2000, 2000, 1), dtype=np.int)
ndvi_hi_max = np.zeros((2000, 2000, 1), dtype=np.int)
ndvi_hi_msk = np.zeros((2000, 2000, 1), dtype=np.int)

for i in tqdm(range(100)):  
    
    i1 = "0" + str(i) if i < 10 else str(i) 

    bbox = BBox(bboxes.iloc[i].geometry, crs=CRS(bboxes.crs))
    id_values = bboxes.eop_index.values[i]

    # get the training set
    get_s2_l2a = SentinelHubInputTask(
        bands_feature=(FeatureType.DATA, 'BANDS'),
        bands_dtype=np.uint16,
        resolution=S2_RESOLUTION,
        maxcc=S2_MAXCC,
        time_difference=dt.timedelta(minutes=S2_TIME_DELTA),
        data_collection=DataCollection.SENTINEL2_L2A,
        additional_data=[(FeatureType.MASK, 'dataMask', 'IS_DATA'),
                         (FeatureType.MASK, 'SCL'),
                         (FeatureType.MASK, 'CLM'),
                         (FeatureType.DATA, 'CLP')],
        max_threads=MAX_THREADS,
        config=sh_config
    )
    
    s2_l2a_eop = get_s2_l2a.execute(bbox=bbox, time_interval = S2_TIME_INTERVAL)
    
    #get the number of acquisitions
    n_imgs = s2_l2a_eop.data['BANDS'].shape[0]
    print("box", i, "downloaded -",s2_l2a_eop.data['BANDS'][...,3].shape, end = ' .. ')
    
    #definition of some band used in the processing
    vis_factor = 3.5
    norm_factor = s2_l2a_eop.scalar['NORM_FACTORS']
    b4 = s2_l2a_eop.data['BANDS'][...,3] * norm_factor[:,None]
    b8 = s2_l2a_eop.data['BANDS'][...,7] * norm_factor[:,None]
    isdata = s2_l2a_eop.mask['IS_DATA'][...,0]
    clm = s2_l2a_eop.mask['CLM'][...,0]
    clp = s2_l2a_eop.data['CLP'][...,0]
    scl = s2_l2a_eop.mask['SCL'][...,0]
    
    #valid = ~isdata | clm | (clp > MIN_CLOUD_PROB)
    #valid = ~isdata | clm | (scl < 4)|(scl > 6)| (clp > MIN_CLOUD_PROB)
    #valid = ~isdata | clm | (clp > MIN_CLOUD_PROB)    
    valid = ~isdata | clm 
    
    np.seterr(divide='ignore', invalid='ignore')

    ndvi = np.around(255*b8/(b8+b4))
    ndvi = np.nan_to_num(ndvi, copy=False).astype(np.int)
    
    ndvi_lo = np.zeros((n_imgs, 500, 500), dtype=np.int)
    ndvi_hi = np.zeros((n_imgs, 2000, 2000), dtype=np.int)
    

    # --------------------------------------------------------
    # call interpolation function and store results in ndvi_lo 
    ndvi_lo = interpolate(ndvi, valid, n_imgs)
    print("interpolation", end = ' .. ')

    # --------------------------------------------------------
    # find best reference image
    ref_img = find_ref(clm, n_imgs)
    print("found ref img", ref_img, end = ' .. ')
    
    # --------------------------------------------------------
    # superresolve and coregister
    ndvi_hi = coreg(ndvi_lo, ref_img, n_imgs)
    print("superres", ndvi_hi.shape, end = ' .. ')
     
    # --------------------------------------------------------
    # stdev of ndvi along all images
    ndvi_hi_sd[...,0] = np.std(ndvi_hi, axis=0)
    ndvi_hi_max[...,0] = np.amax(ndvi_hi, axis=0)
    ndvi_hi_msk[...,0]= np.where(ndvi_hi_max[...,0] < THRES, 0, 1)
   
    print("stdev", end = ' .. ')
    
    # --------------------------------------------------------
    #save to numpy compressed and geotiff  
    new_eopatch = EOPatch(bbox=bbox)
    ndvisd = (FeatureType.DATA_TIMELESS, 'ndvi_sd')
    add_feature = AddFeature(ndvisd)
    new_eopatch = add_feature.execute(new_eopatch, ndvi_hi_sd)
    
    task = ExportToTiff((FeatureType.DATA_TIMELESS, 'ndvi_sd'), folder='geotiff/train', band_indices=[0]) #, band_indices=[1]
    task.execute(new_eopatch, filename = "eopatch-" + i1 + ".tiff")
  
    np.savez_compressed("./data/train/" + i1, ndvi_hi_sd[...,0])


    print("saved compressed", end = '\n')

    
print("End of preprocessing for the training set", end = '\n')

### Preprocessing of the test set

In [ ]:
ndvi_hi_sd = np.zeros((2000, 2000, 1), dtype=np.int)
ndvi_hi_max = np.zeros((2000, 2000, 1), dtype=np.int)
ndvi_hi_msk = np.zeros((2000, 2000, 1), dtype=np.int)

for i in tqdm(range(1,26)):  
    
    i1 = "0" + str(i) if i < 10 else str(i) 
    i2 = "0" + str(i-1) if i-1 < 10 else str(i-1)
    load = LoadTask(path=EOPATCHES_TEST_PATH)

    #load the test set from the local folder
    eops_test = sorted(os.listdir(f'{EOPATCHES_PATH}/test/'))
    s2_l2a_eop = load.execute(eopatch_folder = f'eopatch-' + i1)  
    bbox = s2_l2a_eop.bbox
    
    #get the number of acquisitions
    n_imgs = s2_l2a_eop.data['BANDS'].shape[0]
    print("box", i, "downloaded -",s2_l2a_eop.data['BANDS'][...,3].shape, end = ' .. ')
    
    #definition of some band used in the processing
    vis_factor = 3.5
    norm_factor = s2_l2a_eop.scalar['NORM_FACTORS']
    b4 = s2_l2a_eop.data['BANDS'][...,3] * norm_factor[:,None]
    b8 = s2_l2a_eop.data['BANDS'][...,7] * norm_factor[:,None]
    isdata = s2_l2a_eop.mask['IS_DATA'][...,0]
    clm = s2_l2a_eop.mask['CLM'][...,0]
    clp = s2_l2a_eop.data['CLP'][...,0]
    scl = s2_l2a_eop.mask['SCL'][...,0]
    
    #valid = ~isdata | clm | (clp > MIN_CLOUD_PROB)
    #valid = ~isdata | clm | (scl < 4)|(scl > 6)| (clp > MIN_CLOUD_PROB)
    #valid = ~isdata | clm | (clp > MIN_CLOUD_PROB)    
    valid = ~isdata | clm 
    
    np.seterr(divide='ignore', invalid='ignore')

    ndvi = np.around(255*b8/(b8+b4))
    ndvi = np.nan_to_num(ndvi, copy=False).astype(np.int)
    
    ndvi_lo = np.zeros((n_imgs, 500, 500), dtype=np.int)
    ndvi_hi = np.zeros((n_imgs, 2000, 2000), dtype=np.int)
    
    # --------------------------------------------------------
    # call interpolation function and store results in ndvi_lo 
    
    ndvi_lo = interpolate(ndvi, valid, n_imgs)
    print("interpolation", end = ' .. ')

    # --------------------------------------------------------
    # find best reference image
    
    ref_img = find_ref(clm, n_imgs)
    print("found ref img", ref_img, end = ' .. ')
    
    # --------------------------------------------------------
    # superresolve and coregister
    
    ndvi_hi = coreg(ndvi_lo, ref_img, n_imgs)    
    print("superres", ndvi_hi.shape, end = ' .. ')

    # --------------------------------------------------------
    # stdev of ndvi along all images
    
    ndvi_hi_sd[...,0] = np.std(ndvi_hi, axis=0)
    ndvi_hi_max[...,0] = np.amax(ndvi_hi, axis=0)
    ndvi_hi_msk[...,0]= np.where(ndvi_hi_max[...,0] < THRES, 0, 1)
    print("stdev and mean", end = ' .. ')
    
    # --------------------------------------------------------
    #save to numpy compressed and geotiff  
    
    new_eopatch = EOPatch(bbox=bbox)
    ndvisd = (FeatureType.DATA_TIMELESS, 'ndvi_sd')
    add_feature = AddFeature(ndvisd)
    new_eopatch = add_feature.execute(new_eopatch, ndvi_hi_sd)
    
    task = ExportToTiff((FeatureType.DATA_TIMELESS, 'ndvi_sd'), folder='geotiff/test', band_indices=[0]) #, band_indices=[1]
    task.execute(new_eopatch, filename = "eopatch-" + i1 + ".tif")
  
    np.savez_compressed("./data/test/" + i2, ndvi_hi_sd[...,0])


    print("saved compressed", end = '\n')

    
print("End of preprocessing for the test set", end = '\n')